In [ ]:
# --- CELDA 1: Dependencias y librerías ---
import os
import requests
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\valec\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ---- CELDA 2: Tu API KEY segura (NO subas a github ni la compartas)
# Reemplazá por tu key real, o lee de una variable segura/entorno

os.environ["GROQ_API_KEY"] = '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'

In [3]:
# ---- CELDA 3: Creá corpus y embeddings ----
corpus = [
    "La ley nacional de tránsito establece que el uso de casco es obligatorio en motocicletas en Argentina.",
    "El salario mínimo en Argentina en junio de 2024 es de 234,315 pesos según el Boletín Oficial.",
    "Para recuperar una contraseña en el sistema universitario, se debe acceder a la opción 'Olvidé mi contraseña' y seguir los pasos.",
    "El error estándar se calcula como la raíz cuadrada de la varianza sobre el tamaño de la muestra.",
    "El aprendizaje profundo ha revolucionado el diagnóstico médico por imágenes.",
    "La inscripción a materias en la facultad cierra generalmente a mediados de marzo y agosto.",
]

EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer_emb = AutoTokenizer.from_pretrained(EMBEDDING_MODEL)
model_emb = AutoModel.from_pretrained(EMBEDDING_MODEL)

def compute_embedding(text):
    encoded_input = tokenizer_emb(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model_emb(**encoded_input)
    embeddings = model_output.last_hidden_state
    mask = encoded_input['attention_mask'].unsqueeze(-1).expand(embeddings.size()).float()
    sum_embeddings = torch.sum(embeddings * mask, 1)
    sum_mask = torch.clamp(mask.sum(1), min=1e-9)
    return (sum_embeddings / sum_mask).squeeze().numpy()

chunk_embeddings = np.vstack([compute_embedding(ch) for ch in corpus])

c:\Users\valec\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\valec\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not insta

In [4]:
# ---- CELDA 4: Definí tu pregunta y hacé retrieval semántico ----
query = "¿Cuánto es el salario mínimo actual en Argentina?"
query_emb = compute_embedding(query)
sims = cosine_similarity([query_emb], chunk_embeddings)[0]
top_indexes = sims.argsort()[-3:][::-1]  # Top 3 relevantes

print("Chunks recuperados:")
for i in top_indexes:
    print(f"{corpus[i]}\n(Similitud: {sims[i]:.2f})\n")

context = "\n".join([corpus[i] for i in top_indexes])
prompt = f"""Contexto:
{context}
---
Pregunta: {query}
Respondé de forma precisa y citando la evidencia textual relevante cuando corresponda. Si el contexto no contiene la respuesta, indicá 'No disponible en los textos recuperados'.
"""

print("\n=== Prompt para LLM via Groq ===\n")
print(prompt)

Chunks recuperados:
El salario mínimo en Argentina en junio de 2024 es de 234,315 pesos según el Boletín Oficial.
(Similitud: 0.76)

El error estándar se calcula como la raíz cuadrada de la varianza sobre el tamaño de la muestra.
(Similitud: 0.51)

La ley nacional de tránsito establece que el uso de casco es obligatorio en motocicletas en Argentina.
(Similitud: 0.49)


=== Prompt para LLM via Groq ===

Contexto:
El salario mínimo en Argentina en junio de 2024 es de 234,315 pesos según el Boletín Oficial.
El error estándar se calcula como la raíz cuadrada de la varianza sobre el tamaño de la muestra.
La ley nacional de tránsito establece que el uso de casco es obligatorio en motocicletas en Argentina.
---
Pregunta: ¿Cuánto es el salario mínimo actual en Argentina?
Respondé de forma precisa y citando la evidencia textual relevante cuando corresponda. Si el contexto no contiene la respuesta, indicá 'No disponible en los textos recuperados'.



In [5]:
# ---- CELDA 5: Llama 3 via Groq API ----
def groq_llm(prompt,
             model="llama3-70b-8192", # o llama3-8b-8192 según disponibilidad
             max_tokens=250, temperature=0.1,
             system_message="Sos un asistente experto en IA. Respondé en español."):
    api_url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {os.environ['GROQ_API_KEY']}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": None
    }
    response = requests.post(api_url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        print(response.text)
        raise Exception("Error en la API de Groq")

In [ ]:
# ---- CELDA 6: Ejecutar el LLM sobre tu prompt RAG ----
respuesta = groq_llm(prompt)
print("\n=== Respuesta Llama 3 (Groq) ===\n")
print(respuesta)


=== Respuesta Llama 3 (Groq) ===

Según el contexto, el salario mínimo actual en Argentina es de 234,315 pesos, según el Boletín Oficial.
